In [1]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import plotly.graph_objects as go
import os
from policyengine_core.charts import *

/Users/pavelmakarchuk/anaconda3/envs/pe/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
reform_static = Reform.from_dict(
    {"gov.contrib.congress.golden.fisc_act.in_effect": {"2026-01-01.2100-12-31": True}},
    country_id="us",
)
reform_dynamic = Reform.from_dict(
    {
        "gov.simulation.labor_supply_responses.elasticities.income": {
            "2024-01-01.2100-12-31": -0.05
        },
        "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.1": {
            "2024-01-01.2100-12-31": 0.31
        },
        "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.2": {
            "2024-01-01.2100-12-31": 0.28
        },
        "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.3": {
            "2024-01-01.2100-12-31": 0.27
        },
        "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.4": {
            "2024-01-01.2100-12-31": 0.27
        },
        "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.5": {
            "2024-01-01.2100-12-31": 0.25
        },
        "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.6": {
            "2024-01-01.2100-12-31": 0.25
        },
        "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.7": {
            "2024-01-01.2100-12-31": 0.22
        },
        "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.8": {
            "2024-01-01.2100-12-31": 0.22
        },
        "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.9": {
            "2024-01-01.2100-12-31": 0.22
        },
        "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.secondary": {
            "2024-01-01.2100-12-31": 0.27
        },
        "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.10": {
            "2024-01-01.2100-12-31": 0.22
        },
        "gov.contrib.congress.golden.fisc_act.in_effect": {
            "2026-01-01.2100-12-31": True
        }
    },
    country_id="us",
)

In [3]:
def calculate_budgetary_impact(reform, year):
    baseline = Microsimulation(dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")
    baseline_income = baseline.calculate("income_tax", map_to="household", period=year).sum()

    reformed = Microsimulation(reform=reform, dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")
    reformed_income = reformed.calculate("income_tax", map_to="household", period=year).sum()

    impact = reformed_income - baseline_income
    return impact 

In [4]:
years = range(2026, 2036)  # 2026 to 2035 inclusive
df = pd.DataFrame(index=years, columns=["Static Impact", "Dynamic Impact"])

In [5]:
# Calculate and print impacts one by one
print("Year | Static Impact | Dynamic Impact")
print("-" * 45)

for year in years:
    # Calculate static impact
    print(f"Calculating static impact for {year}...", end="")
    static_impact = calculate_budgetary_impact(reform_static, year)
    df.loc[year, "Static Impact"] = static_impact
    print(f" Done: ${static_impact:,.0f}")
    
    # Calculate dynamic impact
    print(f"Calculating dynamic impact for {year}...", end="")
    dynamic_impact = calculate_budgetary_impact(reform_dynamic, year)
    df.loc[year, "Dynamic Impact"] = dynamic_impact
    print(f" Done: ${dynamic_impact:,.0f}")
    
    # Print the results for this year
    print(f"{year} | ${static_impact:,.0f} | ${dynamic_impact:,.0f}")
    print("-" * 45)
    
    # Optional: Clear memory after each year
    import gc
    gc.collect()

Year | Static Impact | Dynamic Impact
---------------------------------------------
Calculating static impact for 2026... Done: $-203,199,996,035
Calculating dynamic impact for 2026...

KeyboardInterrupt: 

In [ ]:
# Save results to CSV to avoid losing data
df.to_csv("fisc_act_10_year_impact.csv")

# Display the final DataFrame
print("\nFinal results:")
df

In [ ]:
# Visualize the 10-year impact
fig = go.Figure()

fig.add_trace(go.Bar(
    x=df.index,
    y=df["Static Impact"],
    name="Static Impact"
))

fig.add_trace(go.Bar(
    x=df.index,
    y=df["Dynamic Impact"],
    name="Dynamic Impact with Behavioral Responses"
))

fig.update_layout(
    title="10-Year Budgetary Impact of FISC Act",
    xaxis_title="Year",
    yaxis_title="Impact ($)",
    barmode="group"
)

fig.show()